# Spark Connect

## Мотивация

В Apache Spark 3.4.0 появилась технология **Spark Connect**, которая позволяет разделить драйвер на две части: клиент и сервер. Общение между клиентом и сервером осуществляется при помощи [gRPC](https://ru.wikipedia.org/wiki/GRPC) запросов. На данный момент доступно лишь использование **DataFrame API**.

### Как это работает

Обработка запроса в оптимизаторе **Catalyst** проходит четыре стадии:

- Unresolved Logical Plan,
- Logical Plan,
- Optimized Logical Plan,
- Physical Plan.

Разбор запроса до стадии `Unresolved Logical Plan` можно выполнить на клиенте, а остальные стадии будет выполнять сервер. При этом клиент может использовать любой язык программирования, если он способен:

- сформировать Unresolved Logical Plan,
- отправить Unresoled Logical Plan серверу по протоколу [gRPC](https://ru.wikipedia.org/wiki/GRPC).

Таким образом, экосистема **Spark** становится еще более доступной.

Для работы с **Spark Connect** на клиенте необходимо установить библиотеки для работы с `gRPC`:

In [ ]:
! pip install grpcio==1.59.0 grpcio-status==1.59.0

## Запуск клиента

In [ ]:
from pyspark.sql import SparkSession

**Spark Connect** нельзя запустить одновременно с приложением Spark запущенном в локальном режиме, поэтому необходимо проконтролировать, что локальной версии Spark нет.

Следующая команда запустит Spark в локальном режиме или подключится к приложению Spark, запущенному в локальном режиме, и сразу остановит его:

In [ ]:
SparkSession.builder.master("local").getOrCreate().stop()

Сейчас можно подключиться к запущенному приложению Spark на удаленном сервере:

In [ ]:
spark = (
    SparkSession
        .builder
        .remote("sc://connect:15002")
        .getOrCreate()
)

In [ ]:
df = spark.sql("select 'Hello, Spark Connect!' as message")

In [ ]:
df.show(1, False)

## Запуск сервера

Приложение Spark запущено на `yarn` в docker сервисе под названием `connect`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps connect

Учитывая, что сервер запущен в сервисе `connect`, Spark UI также доступен в сервисе `connect`. Не смотря на эту особенность, порт 4040 прокинут на локальную машину и **Spark UI** доступен по адресу [localhost:4040](http://localhost:4040).

Для запуска **Spark Connect** сервера необходимо запустить `$SPARK_HOME/sbin/start-connect-server.sh`:

In [ ]:
! ls -l $SPARK_HOME/sbin/start-connect-server.sh

Сервис `connect` автоматически запускает **Spark Connect** при старте:

In [ ]:
! source ~/.bash_aliases && \
docker compose top connect

Входной точкой сервиса `connect` является файл `/usr/local/bin/entrypoint`, который активирует `start-connect-server.sh`:

In [ ]:
! source ~/.bash_aliases && HOST=connect execute \
grep 'start-connect-server.sh' /usr/local/bin/entrypoint

Параметры запуска приложения передаются через `$SPARK_HOME/conf/spark-defaults.conf`:

In [ ]:
! source ~/.bash_aliases && HOST=connect execute \
cat $SPARK_HOME/conf/spark-defaults.conf

## Остановка клиента

При вызове `SparkSession#stop` остановится только клиент, сервер продолжит обрабатывать запросы от других клиентов:

In [ ]:
spark.stop()

Доступность сервера можно проверить, если запустить нового клиента:

In [ ]:
spark = (
    SparkSession
        .builder
        .remote("sc://connect:15002")
        .getOrCreate()
)

In [ ]:
spark.sql("select 'Hello, Spark Connect!' as message").show(1, False)

In [ ]:
spark.stop()

Если открыть [Spark UI](http://localhost:4040), то можно увидеть список запросов от всех предыдущих приложений.

## Остановка сервера

Остановка сервера осуществляется при помощи скрипта `$SPARK_HOME/sbin/stop-connect-server.sh`:

In [ ]:
! ls -l $SPARK_HOME/sbin/stop-connect-server.sh

In [ ]:
! source ~/.bash_aliases && HOST=connect execute \
$SPARK_HOME/sbin/stop-connect-server.sh

После остановки сервера невозможно запустить нового клиента. Попробуйте запустить код (приложенение Spark просто повиснет):

```python
spark = (
    SparkSession
        .builder
        .remote("sc://connect:15002")
        .getOrCreate()
)
```

Для восстановления сервера необходимо перезапустить сервис `connect`:

In [ ]:
! source ~/.bash_aliases && \
docker compose restart connect

Необходимо дождаться пока в колонке **STATUS** появится значение `healthy`:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps connect

Spark Connect сервер также остановится, если просто остановить сервис `connect`:

In [ ]:
! source ~/.bash_aliases && \
docker compose stop connect

## Вывод

Архитектура **Catalyst** оказалась настолько удачной, что открыла дверь для **Spark Connect**. Теперь можно запустить драйвер на мощной машине в облаке, а программисты могут использовать менее производительное аппаратное обеспечение, не теряя при этом эффективности. Spark Connect достаточно молодая технология, поэтому не получила пока широкого распространения, но в недалеком будущем программисты смогут использовать любой язык для работы со Spark, т.к. для поддержки Spark Connect на клиенте необходимо разработать разбор запросов, а также реализовать несколько gRPC [контрактов](https://github.com/apache/spark/tree/c47a9205b2de1c0638e3824564f0d09bad3f4b24/connector/connect/common/src/main/protobuf/spark/connect).

## Задание

1. Запустить Spark Connect сервер со следующей конфигурацией:
    - локальный режим на 4 потока,
    - `1512MB` памяти,
    - на порту [15020](https://spark.apache.org/docs/latest/configuration.html#spark-connect).
2. Подключиться к Spark Connect серверу.
3. Проверить настройки через Spark UI. Почему объем доступной памяти меньше `1512M`?

### Ответы

1. Запустить Spark Connect сервер со следующей конфигурацией:
    - локальный режим на 4 потока,
    - `1512MB` памяти,
    - на порту [15020](https://spark.apache.org/docs/latest/configuration.html#spark-connect).

<details>
    <summary>Ответ</summary>

1. Открыть файл `conf/spark-connect.conf` в проекте. Через текущий ноутбук будет сложно отредактировать этот файл.
2. Добавить конфиги:

```properties
spark.driver.memory=1512m
spark.connect.grpc.binding.port=15020
```

3. Открыть файл `bin/spark-connect-entrypoint` в проекте. Через текущий ноутбук будет сложно отредактировать этот файл.
Заменить `yarn` на `local[4]`:

```bash
function start_spark_connect() {
    $SPARK_HOME/sbin/start-connect-server.sh --packages org.apache.spark:spark-connect_2.12:3.5.0 --master local[4]
    ...
```

4. Запустить сервис `connect`:

```bash
! source ~/.bash_aliases && \
docker compose start connect && \
```

5. Команда `docker compose ps connect` ожидаемо покажет значение `unhealty`, т.к. docker ожидает, что **Spark Connect** запущен на порту `15002`

```bash
! source ~/.bash_aliases && \
docker compose start connect && \
docker compose ps connect
```
</details>

2. Подключиться к Spark Connect серверу.

<details>
    <summary>Ответ</summary>

Если команда ниже повиснет, то рекомендуется перезапустить ноутбук:
- в главном меню выбрать пункт `Kernel`,
- выбрать `Restart Kernel...`,
- подтвердить перезапуск.

```python
spark = (
    SparkSession
        .builder
        .remote("sc://connect:15020")
        .getOrCreate()
)
```
</details>

3. Проверить настройки через Spark UI.

<details>
    <summary>Ответ</summary>

1. Открыть http://localhost:4040/executors
2. Всего доступно ядер (cores): 4
3. Всего памяти доступно `727.2M`, но это всего лишь `60%` от всей памяти (M-регион), поэтому всего памяти выделено:

$$
\frac{727.2MB} {0.6 (spark.memory.fraction)} + 300MB (зарезервировано) = 1512M
$$

Полученное значение соответствует запрошенному объему (значение `spark.driver.memory` в `conf/spark-connect.conf`).

</details>

